### Some helper functions:

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os.path
import matplotlib.pyplot as plt
import matplotlib

def prepare(f, v=False):
    df = pd.read_csv(f, comment='#')
    try:
        datetimes = pd.to_datetime(df["DATE_TIME"])
        df["DATE_TIME"] = datetimes
    except KeyError:
        df.rename(columns={"START_DATE": "DATE_TIME"}, inplace=True)
        datetimes = pd.to_datetime(df["DATE_TIME"])
        df["DATE_TIME"] = datetimes
    if v:
        df.info()
    return df


def calc_intra_annual(df, mean_col, max_col, min_col):
    mean_list = []
    std_list = []
    max_list = []
    min_list = []

    for d in range(1, 366):
        # print(d)
        mask = ((df["DATE_TIME"].dt.dayofyear == d) & (~df["DATE_TIME"].dt.is_leap_year)) | (
                    (df["DATE_TIME"].dt.dayofyear == d + 1) & (df["DATE_TIME"].dt.dayofyear > 59) & df[
                "DATE_TIME"].dt.is_leap_year) | (
                           (df["DATE_TIME"].dt.dayofyear == d) & (df["DATE_TIME"].dt.dayofyear <= 59) & df[
                       "DATE_TIME"].dt.is_leap_year)
        day_df = df.loc[mask]
        # print(day_df)

        mean = day_df[mean_col].mean()
        mean_list.append(mean)

        std = day_df[mean_col].std()
        std_list.append(std)

        max_list.append(day_df[max_col].max())
        min_list.append(day_df[min_col].min())

    day_list = [x for x in range(1, 366)]
    # print(len(day_list))
    # print(len(mean_list))
    # print(len(std_list))
    upper_list = [sum(x) for x in zip(mean_list, std_list)]
    lower_list = [x[0] - x[1] for x in zip(mean_list, std_list)]

    return day_list, mean_list, upper_list, lower_list, max_list, min_list


def set_plt():
    cm = 1 / 2.54  # centimeters in inches
    plt.figure(figsize=(8.5, 5.5), dpi=80)
    plt.rcParams.update({'font.size': 16})
    plt.tight_layout()


def plot(day_list, mean_list, upper_list=None, lower_list=None, max_list=None, min_list=None, ylim=(-47, +40), ylabel=None, title=None):
    # plt.figure(figsize=(8, 4), dpi=80)
    set_plt()

    plt.plot(day_list, mean_list, color="k")
    """
    if upper_list is not None:
        plt.plot(day_list, upper_list, color="grey")
    if lower_list is not None:
        plt.plot(day_list, lower_list, color="grey")
    """
    plt.fill_between(day_list, upper_list, lower_list, color="lightgrey")
    if max_list is not None:
        # plt.plot(day_list, max_list, color="r")
        plt.plot(day_list, max_list, color="grey")
    if min_list is not None:
        # plt.plot(day_list, min_list, color="r")
        plt.plot(day_list, min_list, color="grey")

    plt.ylim(ylim)
    if ylabel is None:
        ylabel = "Air temperature, degree Celsius"
    plt.ylabel(ylabel)
    plt.xlabel("Day of year")
    # months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', '']
    # plt.xticks(np.linspace(0, 365, 13), months)

    plt.axhline(y=0, color='k', linestyle='-', linewidth=0.5)

    if title is not None:
        plt.title(title)

    plt.show()

### General font settings:

In [2]:
cm = 1 / 2.54  # inches to cm

LEAST_SIZE = 7
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

matplotlib.rc('font', size=SMALL_SIZE)          # controls default text sizes
matplotlib.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
matplotlib.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
matplotlib.rc('xtick', labelsize=LEAST_SIZE)    # fontsize of the tick labels
matplotlib.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
matplotlib.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
matplotlib.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

### Input and output directories:

In [3]:
work_dir = "/home/antonv/data/essd/PREP/READY_V2/"

# fig_dir = os.path.dirname(os.path.realpath(__file__))  # __file__ is undefined in Jupyter Notebook
fig_dir = os.path.join(globals()['_dh'][0], "fig")
# print(fig_dir)

### Settings for Pandas: calendar year -> hydrological year

In [4]:
SHIFT = 2  # to start hydrological year from November (more useful for Lammin Suo)
# SHIFT = 3  # to start hydrological year from October

### Air temperature, intra-annual:

In [ ]:
files = ("meteo_daily_1952-2020_1.csv", )

for f in files:
    
    df = prepare(os.path.join(work_dir, f))
    df = df.loc[df["DATE_TIME"].dt.year > 1966]  # observation timetable was changed starting from 1967
    df["DAILY_AMPL"] = df["MAX_AIR_TEMP"] - df["MIN_AIR_TEMP"]
    
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(17 / 2.54, 6 / 2.54)
    # fig.suptitle('Horizontally stacked subplots')

    day_list, mean_list, upper_list, lower_list, max_list, min_list = calc_intra_annual(df, "MEAN_AIR_TEMP",
                                                                                            "MAX_AIR_TEMP",
                                                                                            "MIN_AIR_TEMP")
    
    ax1.plot(day_list, mean_list, color="k", linewidth=1)
    ax1.fill_between(day_list, upper_list, lower_list, color="lightgrey")
    ax1.plot(day_list, max_list, color="grey", linewidth=1)
    ax1.plot(day_list, min_list, color="grey", linewidth=1)
    ax1.hlines(y=0, xmin=1, xmax=365, linewidth=0.5, color='r')
    ax1.set_ylim([-47, +40])
    ax1.set_xlabel("Day of year")
    ax1.set_ylabel("Air temperature,°C")
    ax1.text(-0.14, 1.15, "(a)", transform=ax1.transAxes, fontsize=MEDIUM_SIZE, va='top', ha='right')
    


    day_list, mean_list, upper_list, lower_list, max_list, min_list = calc_intra_annual(df, "DAILY_AMPL",
                                                                                            "DAILY_AMPL",
                                                                                            "DAILY_AMPL")
    ax2.plot(day_list, mean_list, color="k", linewidth=1)
    ax2.fill_between(day_list, upper_list, lower_list, color="lightgrey")
    ax2.plot(day_list, max_list, color="grey", linewidth=1)
    ax2.plot(day_list, min_list, color="grey", linewidth=1)
    ax2.hlines(y=0, xmin=1, xmax=365, linewidth=0.5, color='r')
    ax2.set_ylim([-2, +35])
    
    ax2.set_xlabel("Day of year")
    ax2.set_ylabel("Amplitude,°C")
    ax2.text(-0.1, 1.15, "(b)", transform=ax2.transAxes, fontsize=MEDIUM_SIZE, va='top', ha='right')
    
    fig.tight_layout()
    
    fig_path = os.path.join(fig_dir, "test_01.png")
    plt.savefig(fig_path, dpi=300, transparent=False, facecolor="white")



### Air temperature, trends:

In [ ]:
files = ("meteo_daily_1952-2020_1.csv", )

tmp = prepare(os.path.join(work_dir, f))
tmp.set_index("DATE_TIME", inplace=True)

# fig, (ax1, ax2) = plt.subplots(2, 1)
fig, ax = plt.subplots()
fig.set_size_inches(17 / 2.54, 10 / 2.54)

def plot_row(tmp, ax, start, end):
    df = tmp.loc[(tmp.index.year >= start) & (tmp.index.year <= end)]

    df["PDD"] = df["MEAN_AIR_TEMP"].clip(lower=0)  # for positive degree-day sums
    df["NDD"] = df["MEAN_AIR_TEMP"].clip(upper=0)  # for negative degree-day sums
    df["NDD"] = abs(df["NDD"])

    df = df.resample("M").sum()
    df = df.shift(SHIFT)
    df = df.resample("A").sum()
    df["HYDRO_YEAR"] = (df.index.year - 1).astype(str) + "/" + df.index.year.astype(str)
    df = df.iloc[1:]  # since first hydro year is truncated (there is no Oct-Dec of 1966 in dataset)
    # print(df)

    # ax.set_xlim([0, len(df["HYDRO_YEAR"])])
    # ax.set_ylim([-0.05, 0.5])
    ax.plot(df["HYDRO_YEAR"], df["PDD"], color="k", linewidth=1, label="positive")
    ax.plot(df["HYDRO_YEAR"], df["NDD"], color="grey", linewidth=1, label="abs(negative)")
    ax.set_ylabel("Degree-day sum, °C×day")
    for tick in ax.get_xticklabels():
        tick.set_rotation(90)

# plot_row(tmp, ax1, 1950, 1966) # observation timetable was changed starting from Jan1967
# plot_row(tmp, ax2, 1967, 2021) # observation timetable was changed starting from Jan1967
plot_row(tmp, ax, 1950, 2021) # observation timetable was changed starting from Jan1967

ax.legend()

# ax2.set_xlabel("Hydrological year")
ax.set_xlabel("Hydrological year")
# for tick in ax2.get_xticklabels()[1::2]:
for tick in ax.get_xticklabels()[1::2]:
    tick.set_visible(False)

fig.tight_layout()   

fig_path = os.path.join(fig_dir, "test_01x.png")
plt.savefig(fig_path, dpi=300, transparent=False, facecolor="white")

### Precipitation, monthly:

In [ ]:
f = "precipitation_daily_1952-2020.csv"

tmp = prepare(os.path.join(work_dir, f))
tmp.set_index("DATE_TIME", inplace=True)
df = tmp.loc[tmp["GAUGE_NO"] == 1]  # the longest time series

df["P_LIQUID"] = np.where(df["SOLID"] == 0, df["PRECIPITATION"], np.nan)
df["P_SOLID"] = np.where(df["SOLID"] == 1, df["PRECIPITATION"], np.nan)
df = df.resample('M').sum()
df["MONTH"] = df.index.month

mean = df.groupby(["MONTH"]).mean()
# print(mean)
std = df.groupby(["MONTH"]).std()
# print(std)
max_df = df.groupby(["MONTH"]).max()
# print(max_df)
min_df = df.groupby(["MONTH"]).min()  # contains zeroes only, useless
# print(min_df)
fig, ax = plt.subplots(2, 1)
fig.set_size_inches(17 / 2.54, 10 / 2.54)

def plot_prec(ax, mean, std, max_df, col_name):
    ax.plot(mean.index, mean[col_name], color="k", linewidth=1)
    upper_list = mean + std
    # upper_list.replace(0, np.nan, inplace=True)
    lower_list = mean - std
    lower_list = lower_list.clip(lower=0)
    ax.fill_between(mean.index, upper_list[col_name], lower_list[col_name], color="lightgrey")
    ax.plot(mean.index, max_df[col_name], color="grey", linewidth=1)
    # ax.set_ylim([-47, +40])
    ax.text(-0.14, 1.15, "(a)", transform=ax1.transAxes, fontsize=MEDIUM_SIZE, va='top', ha='right')

plot_prec(ax[0], mean, std, max_df, "P_LIQUID")
plot_prec(ax[1], mean, std, max_df, "P_SOLID")

ax[-1].set_xlabel("Month")
months = ("Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec")
for a in ax:
    plt.sca(a)
    plt.xticks(np.linspace(1, 12, num=len(months)), months)

ax[0].text(-0.1, 1.15, "(a)", transform=ax[0].transAxes, fontsize=MEDIUM_SIZE, va='top', ha='right')
ax[0].set_ylabel("Liquid precipitation\nsum, mm")
ax[1].text(-0.1, 1.15, "(b)", transform=ax[1].transAxes, fontsize=MEDIUM_SIZE, va='top', ha='right')
ax[1].set_ylabel("Solid precipitation\nsum, mm")

fig.tight_layout()

fig_path = os.path.join(fig_dir, "test_02x.png")
plt.savefig(fig_path, dpi=300, transparent=False, facecolor="white")

### Precipitation:

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

f = "precipitation_daily_1952-2020.csv"

tmp = prepare(os.path.join(work_dir, f))
tmp.set_index("DATE_TIME", inplace=True)
df = tmp.loc[tmp["GAUGE_NO"] == 1]

df["P_TOTAL"] = df["PRECIPITATION"]
df["P_LIQUID"] = np.where(df["SOLID"] == 0, df["PRECIPITATION"], np.nan)
df["P_SOLID"] = np.where(df["SOLID"] == 1, df["PRECIPITATION"], np.nan)
df = df.resample('M').sum()
# print(df)

# Shifting all the columns to obtain hydrological years instead of calendar:
df["P_TOTAL"] = df["P_TOTAL"].shift(SHIFT)
df["P_LIQUID"] = df["P_LIQUID"].shift(SHIFT)
df["P_SOLID"] = df["P_SOLID"].shift(SHIFT)
# print(df)

df = df.resample('Y').sum()
df["HYDRO_YEAR"] = (df.index.year - 1).astype(str) + "/" + df.index.year.astype(str)
df = df.iloc[1:]  # since first hydro year is truncated (there is no Oct-Dec of 1951 in dataset)

df.loc[df["P_LIQUID"] == 0, "P_LIQUID"] = np.nan
df.loc[df["P_SOLID"] == 0, "P_SOLID"] = np.nan

df["SP"] = df["P_SOLID"] / df["P_TOTAL"]  # S/P ratio
df.loc[df["SP"] == 0, "SP"] = np.nan
# print(df)

# fig, ax = plt.subplots()
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1)
fig.set_size_inches(17 / 2.54, 17 / 2.54)

ax1.set_xlim([0, len(df["HYDRO_YEAR"])])
ax1.plot(df["HYDRO_YEAR"], df["P_TOTAL"], color="k", linewidth=1)
ax1.set_ylabel("Total precipitation, mm")

ax2.set_xlim([0, len(df["HYDRO_YEAR"])])
ax2.set_ylim([-10, 1200])
ax2.plot(df["HYDRO_YEAR"], df["P_LIQUID"], color="k", linewidth=1)
ax2.set_ylabel("Liquid precipitation, mm")

ax3.set_xlim([0, len(df["HYDRO_YEAR"])])
ax3.set_ylim([-10, 750])
ax3.plot(df["HYDRO_YEAR"], df["P_SOLID"], color="k", linewidth=1)
ax3.set_ylabel("Solid precipitation, mm")

ax4.set_xlim([0, len(df["HYDRO_YEAR"])])
ax4.set_ylim([-0.05, 0.5])
ax4.plot(df["HYDRO_YEAR"], df["SP"], color="k", linewidth=1)
ax4.set_ylabel("S/P ratio")
ax4.set_xlabel("Hydrological year")

for tick in ax1.get_xticklabels()[::1]:
    tick.set_visible(False)
for tick in ax2.get_xticklabels()[::1]:
    tick.set_visible(False)
for tick in ax3.get_xticklabels()[::1]:
    tick.set_visible(False)
for tick in ax4.get_xticklabels():
    tick.set_rotation(90)
for tick in ax4.get_xticklabels()[0::2]:
    tick.set_visible(False)

ax1.text(-0.1, 1.15, "(a)", transform=ax1.transAxes, fontsize=MEDIUM_SIZE, va='top', ha='right')
ax2.text(-0.1, 1.15, "(b)", transform=ax2.transAxes, fontsize=MEDIUM_SIZE, va='top', ha='right')
ax3.text(-0.1, 1.15, "(c)", transform=ax3.transAxes, fontsize=MEDIUM_SIZE, va='top', ha='right')
ax4.text(-0.1, 1.15, "(d)", transform=ax4.transAxes, fontsize=MEDIUM_SIZE, va='top', ha='right')

fig.subplots_adjust(bottom=0.4)
fig.tight_layout()

fig_path = os.path.join(fig_dir, "test_02.png")
plt.savefig(fig_path, dpi=300, transparent=False, facecolor="white")

### Snow surveys:

In [ ]:
f = "meteo_daily_1952-2020_1.csv"

SHIFT = 2  # to start the hydrological year in November

###################

df = prepare(os.path.join(work_dir, f))
# df.set_index("DATE_TIME", inplace=True)

day_list, mean_list, upper_list, lower_list, max_list, min_list = calc_intra_annual(df, "SNOW_THICKNESS",
                                                                                            "SNOW_THICKNESS",
                                                                                            "SNOW_THICKNESS")   
fig, (ax1, ax2, ax3) = plt.subplots(3, 1)
fig.set_size_inches(17 / 2.54, 17 / 2.54)

ax1.plot(day_list, mean_list, color="k", linewidth=1)
ax1.fill_between(day_list, upper_list, lower_list, color="lightgrey")
ax1.plot(day_list, max_list, color="grey", linewidth=1)
ax1.plot(day_list, min_list, color="grey", linewidth=1)
ax1.set_ylim([-2, 130])
    
ax1.set_xlabel("Day of year")
ax1.set_ylabel("Thickness of the snow cover, cm")

#######################################################

df = prepare(os.path.join(work_dir, f))
df.set_index("DATE_TIME", inplace=True)

df = df.loc[df.index.month < 6]
df = df.resample("A")["SNOW_THICKNESS"].agg(["max"])
df["YEAR"] = df.index.year
# df = df.loc[df["YEAR"] > 1985]
df.dropna(inplace=True)
# print("LENGTH", len(df.index))
df["HYDRO_YEAR"] = (df.index.year - 1).astype(str) + "/" + df.index.year.astype(str)

ax2.plot(df["HYDRO_YEAR"], df["max"], color="k", linewidth=1, label="Thickness")

df = df.loc[df["YEAR"] > 1988]
df.dropna(inplace=True)
fit = np.polyfit(df["YEAR"], df["max"], 1)
fit_fn = np.poly1d(fit)
print("Trend eqn:", fit_fn)
df["predicted"] = df["YEAR"] * fit[0] + fit[1]

ax2.plot(df["HYDRO_YEAR"], df["predicted"], color="grey", linewidth=1, label="Trend since 1988")
ax2.set_ylabel("Maximum thickess of the\nsnow cover in spring, cm")
# ax2.set_xlim([0, len(df["HYDRO_YEAR"])])
for tick in ax2.get_xticklabels():
    tick.set_rotation(90)
for tick in ax2.get_xticklabels()[1::2]:
    tick.set_visible(False)

##############################################################################

df = prepare(os.path.join(work_dir, f))
df.set_index("DATE_TIME", inplace=True)

df["DAY"] = np.where(df["SNOW_THICKNESS"] > 0, 1, 0)

df = df.resample("M")["DAY"].agg(["sum"])
df["DAY_SHIFT"] = df["sum"].shift(SHIFT)

df = df.resample("A")["DAY_SHIFT"].agg(["sum"])
df["sum"] = np.where(df["sum"] == 0, np.nan, df["sum"])
df["HYDRO_YEAR"] = (df.index.year - 1).astype(str) + "/" + df.index.year.astype(str)
df = df.iloc[5:]
# print("LENGTH", len(df.index))

ax3.plot(df["HYDRO_YEAR"], df["sum"], color="k", linewidth=1, label="")
ax3.set_xlabel("Hydrological year")
ax3.set_ylabel("Period with a snow cover, days")
# ax3.set_xlim([0, len(df["HYDRO_YEAR"])])
ax3.set_ylim([-1, 200])
for tick in ax3.get_xticklabels():
    tick.set_rotation(90)
for tick in ax3.get_xticklabels()[1::2]:
    tick.set_visible(False)

ax1.text(-0.1, 1.15, "(a)", transform=ax1.transAxes, fontsize=MEDIUM_SIZE, va='top', ha='right')
ax2.text(-0.1, 1.15, "(b)", transform=ax2.transAxes, fontsize=MEDIUM_SIZE, va='top', ha='right')
ax3.text(-0.1, 1.15, "(c)", transform=ax3.transAxes, fontsize=MEDIUM_SIZE, va='top', ha='right')
    
fig.tight_layout()

fig_path = os.path.join(fig_dir, "test_03.png")
plt.savefig(fig_path, dpi=300, transparent=False, facecolor="white")

### Peat temperatures:

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1)
fig.set_size_inches(17 / 2.54, 12 / 2.54)

##### Daily series:
f = "peat-temp_daily_1960-2020.csv"

tmp = prepare(os.path.join(work_dir, f))
tmp.set_index("DATE_TIME", inplace=True)

tmp = tmp.loc[tmp["QC"] != 0]
tmp = tmp.loc[tmp["DEPTH"] == 0]
tmp = tmp.loc[tmp["OBS_TYPE"] == "avg"]
tmp = tmp.loc[tmp.index.year < 1976]  # 1976 is non-complete

for site in (1, ):  # site 2 was observed only during summertime, we won't use it
    df = tmp.loc[tmp["SITE"] == site]
    df = df.resample("A")["PEAT_TEMP"].agg(["mean"])
    # print(df)
    ax1.plot(df.index.year, df["mean"], color="k", linewidth=1, label="Site %s" % site)
    ax1.xaxis.set_ticks(np.arange(df.index.year.min(), df.index.year.max()+1, 1))

ax1.set_ylabel("Mean daily temperature of the\npeat surface,°C")
    
##### Hourly series:

f = "peat-temp_hourly_1976-2020.csv"

tmp = prepare(os.path.join(work_dir, f))
tmp.set_index("DATE_TIME", inplace=True)

tmp = tmp.loc[tmp["QC"] != 0]
tmp = tmp.loc[tmp["DEPTH"] == 0]
tmp = tmp.loc[tmp.index.hour == 15]

df = tmp.loc[tmp["SITE"] == 1]
# print(df)
df = df.resample("A")["PEAT_TEMP"].agg(["mean", "max"])
# print(df)
ax2.plot(df.index.year, df["mean"], color="k", linewidth=1)
# ax.plot(df.index.year, df["max"], color="k", linewidth=1, label="Site %s" % site)

ax2.set_ylabel("Mean temperature of the\npeat surface at 15:00,°C")
ax2.set_xlabel("Year")

ax1.text(-0.1, 1.15, "(a)", transform=ax1.transAxes, fontsize=MEDIUM_SIZE, va='top', ha='right')
ax2.text(-0.1, 1.15, "(b)", transform=ax2.transAxes, fontsize=MEDIUM_SIZE, va='top', ha='right')

fig.tight_layout()

fig_path = os.path.join(fig_dir, "test_04.png")
plt.savefig(fig_path, dpi=300, transparent=False, facecolor="white")

fit = np.polyfit(df.index.year, df["mean"], 1)
fit_fn = np.poly1d(fit)
print("Trend eqn:", fit_fn)

### Freezing of the peat surface:

In [ ]:
f = "1954-2019_thaw.csv"

fig, (ax1, ax2) = plt.subplots(2, 1)
fig.set_size_inches(17 / 2.54, 12 / 2.54)

tmp = prepare(os.path.join(work_dir, f))
tmp.set_index("DATE_TIME", inplace=True)
tmp.sort_index(inplace=True)

sites = tmp["SITE"].unique()
# print(sites)

for site in sites:
    print("Site:", site)
    df = tmp.loc[tmp["SITE"] == site]
    t = df["TERRAIN"].unique()
    print(t)
    df = df.loc[df["TERRAIN"] == 1]
    df = df.resample("A")["FROZEN_CM"].agg(["max"])
    if site == "скв.108" or site == "скв.105" or site == "скв.107а" or site == "уч.4":  # OH, NO! the series is interrupted after 1991!
        ax1.plot(df.index.year, df["max"], color="k", linewidth=1)
        ax1.xaxis.set_ticks(np.arange(df.index.year.min(), df.index.year.max()+1, 2))
    if site == "скв.103" or site == "106":
        ax2.plot(df.index.year, df["max"], color="k", linewidth=1)
        ax2.xaxis.set_ticks(np.arange(df.index.year.min(), df.index.year.max()+1, 2))
        
for tick in ax1.get_xticklabels():
    tick.set_rotation(90)
for tick in ax2.get_xticklabels():
    tick.set_rotation(90)

for ax in (ax1, ax2):
    ax.set_ylim([-1, 45])

    ax.set_ylabel("Maximum annual depth\nof a frozen layer, cm")
    ax.set_xlabel("Year")

ax1.text(-0.1, 1.15, "(a)", transform=ax1.transAxes, fontsize=MEDIUM_SIZE, va='top', ha='right')
ax2.text(-0.1, 1.15, "(b)", transform=ax2.transAxes, fontsize=MEDIUM_SIZE, va='top', ha='right')

fig.tight_layout()

fig_path = os.path.join(fig_dir, "test_05.png")
plt.savefig(fig_path, dpi=300, transparent=False, facecolor="white")

### Evapotranspiration:

In [ ]:
f = "evaporation_10d_1956-2020.csv"
print(os.path.join(work_dir, f))

fig, (ax1, ax2) = plt.subplots(2, 1)
fig.set_size_inches(17 / 2.54, 12 / 2.54)

tmp = prepare(os.path.join(work_dir, f))

tmp.set_index("DATE_TIME", inplace=True)
tmp.sort_index(inplace=True)

df = tmp.loc[tmp["SITE"] == 1]  # the longest time series

# INTRA_ANNUAL VARIABILITY:

df["MONTH"] = df.index.month
# getting the number of a 10-day period of a month:
df["D"] = 1
df.loc[df.index.day > 10, "D"] = 2
df.loc[df.index.day > 20, "D"] = 3
df["10d"] = (df["MONTH"] - 1) * 3 + df["D"]  # number of a 10-day period within a year
# print(df)

mean_list = []
std_list = []
max_list = []
min_list = []
day_list = []

for i in range(1, 37):  # there are 36 10-day period in a year
    df_d = df.loc[df["10d"] == i]
    
    if not df_d.empty:
        # print(df_d)
        mean_et = df_d["ET"].mean()
        std_et = df_d["ET"].std()
        max_et = df_d["ET"].max()
        min_et = df_d["ET"].min()
    else:
        mean_et = np.nan
        std_et = np.nan
        max_et = np.nan
        min_et = np.nan
        
    mean_list.append(mean_et)
    std_list.append(std_et)
    max_list.append(max_et)
    min_list.append(min_et)
    day_list.append(i)

upper_list = [sum(x) for x in zip(mean_list, std_list)]
lower_list = [x[0] - x[1] for x in zip(mean_list, std_list)]
    
ax1.plot(day_list, mean_list, color="k", linewidth=1)
ax1.fill_between(day_list, upper_list, lower_list, color="lightgrey")
ax1.plot(day_list, max_list, color="grey", linewidth=1)
ax1.plot(day_list, min_list, color="grey", linewidth=1)
ax1.set_ylim([-2, 71])

ax1.set_xlim([9, 33])
ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(1))
months = ["", 'Apr', "", "", 'May', "", "", 'Jun',"", "",  'Jul',"", "",  'Aug',"", "",  'Sep',"", "",  'Oct',"", "",  'Nov', "", ""]
plt.sca(ax1)
plt.xticks(np.linspace(9, 33, num=len(months)), months)
ax1.set_xticks(np.linspace(9, 33, num=len(months)), months)

ax1.set_xlabel("Thirds of the months")
ax1.set_ylabel("Measured evapotranspiration, mm")
# ax1.text(-0.14, 1.15, "(a)", transform=ax1.transAxes, fontsize=MEDIUM_SIZE, va='top', ha='right')    

# INTER-ANNUAL VARIABILITY:
df = df.resample("A")["ET"].agg(["sum"])

df["HYDRO_YEAR"] = (df.index.year - 1).astype(str) + "/" + df.index.year.astype(str)
# print(df)

ax2.plot(df["HYDRO_YEAR"], df["sum"], color="k", linewidth=1)
ax2.set_ylim([-2, 500])
ax2.set_xlabel("Hydrological year")
ax2.set_ylabel("Annual evapotranspiration sum, mm")
for tick in ax2.get_xticklabels():
    tick.set_rotation(90)
for tick in ax2.get_xticklabels()[1::2]:
    tick.set_visible(False)

ax1.text(-0.1, 1.15, "(a)", transform=ax1.transAxes, fontsize=MEDIUM_SIZE, va='top', ha='right')
ax2.text(-0.1, 1.15, "(b)", transform=ax2.transAxes, fontsize=MEDIUM_SIZE, va='top', ha='right')

fig.tight_layout()

fig_path = os.path.join(fig_dir, "test_06.png")
plt.savefig(fig_path, dpi=300, transparent=False, facecolor="white")


### TODO:
- peat temp by depth
- runoff
- GW levels
- GW temperature
- SW levels
- surface elevation